In [7]:
def compose(f,g):
   return lambda x:f(g(x))
   

def square(x):
   return x * x

def increase(x):
   return x + 1

def double(x):
   return x * 2

m = compose(square,increase)
print(m(5))
m = compose(double,compose(increase,square))
print(m(5)) # Kết quả là 52 do (5 * 5 + 1) * 2




36
52


In [8]:
def f(x):

   def g(y):

      return y**x

   return g

square = f(2)

print(square(3)) # kết quả là 9

cube = f(3)

print(cube(2)) # kết quả là 8

9
8


In [9]:
from functools import reduce
def compose(*g):
   def h(args):
      return reduce(lambda x,y: y(x),reversed(g),args)
   return h

def square(x):
   return x * x

def increase(x):
   return x + 1

def double(x):
   return x * 2

m = compose(square,increase,double)
print(m(5))

121


In [10]:
def f(x):

    return lambda y:y**x
 

square = f(2)

print(square(3)) # kết quả là 9

cube = f(3)

print(cube(2)) # kết quả là 8

9
8


In [11]:
def lessThan(a:list,b:int):
    # return [x for x in a if x < b]
    return list(filter(lambda x,d=b: x < d , a))

def lstSquare(a:int):
    # return [x**2 for x in range(1,a+1)]
    if(a == 1): return [1]
    else: 
        k = lstSquare(a-1)
        k.append(a**2)
        return k
def flatten(a):
    if (a == []): return []
    # return a[0]+ flatten(a[1::1])
    from functools import reduce
    return reduce(lambda x,y: x+y,a)

def dist(a,b):
    # if (a == []): return []
    # return [(a[0],b)]+ dist(a[1::1],b)
    return list(map(lambda x,d=b:(x,d),a))
lessThan([1,2,3,4,5],4)
# lstSquare(3)
flatten([[1,2,3],[4,5],[6,7]])
# dist([1,2,3],4)

def powGen(x):
    def inner(y):
        return y**x
    return inner
	
cube = powGen(3)
print(cube(2))




8


In [18]:
class Eval: 
    def visit(self,ctx):
        return ctx.accept(self)
    def visitIntLit(self,ctx):
        return ctx.value
    def visitFloatLit(self,ctx):
        return ctx.value
    def visitBinExp(self,ctx):
        lctx = self.visit(ctx.left)
        rctx = self.visit(ctx.right)
        if ctx.op == '+':
            return lctx + rctx
        elif ctx.op == '-':
            return lctx - rctx
        elif ctx.op == '*':
            return lctx * rctx
        else:
            return lctx / rctx
    def visitUnExp(self,ctx):
        e = self.visit(ctx.operand)
        return -e if ctx.op == '-' else e
class PrintPrefix:
    def visit(self,ctx):
        return ctx.accept(self)
    def visitIntLit(self,ctx):
        return str(ctx.value)
    def visitFloatLit(self,ctx):
        return str(ctx.value)
    def visitBinExp(self,ctx):
        left = self.visit(ctx.left)
        right = self.visit(ctx.right)
        if ctx.op == '+':
            return '+ ' + left + ' ' + right
        elif ctx.op == '-':
            return '- ' + left + ' ' + right
        elif ctx.op == '*':
            return '* ' + left + ' ' + right
        else:
            return '/ ' + left + ' ' + right
    def visitUnExp(self,ctx):
        e = self.visit(ctx.operand)
        return '-. ' + e if ctx.op == '-' else '+. ' + e
class PrintPostfix: 
    def visit(self,ctx):
        return ctx.accept(self)
    def visitIntLit(self,ctx):
        return str(ctx.value)
    def visitFloatLit(self,ctx):
        return str(ctx.value)
    def visitBinExp(self,ctx):
        left = self.visit(ctx.left)
        right = self.visit(ctx.right)
        if ctx.op == '+':
            return left + ' ' + right + ' +'
        elif ctx.op == '-':
            return left + ' ' + right + ' -'
        elif ctx.op == '*':
            return left + ' ' + right + ' *'
        else:
            return left + ' ' + right + ' /'
    def visitUnExp(self,ctx):
        e = self.visit(ctx.operand)
        return e + ' -.' if ctx.op == '-' else  e + ' +. '
from abc import ABC
class Exp(ABC):pass
class BinExp(Exp):
    def __init__(self,o1,op,o2):
        self.left = o1
        self.op = op
        self.right = o2
    def accept(self,v): return v.visitBinExp(self)
class UnExp(Exp): 
    def __init__(self,op,o1):
        self.op = op
        self.operand = o1
    def accept(self,v): return v.visitUnExp(self)
class IntLit(Exp):
    def __init__(self,v):
        self.value = v
    def accept(self,v): return v.visitIntLit(self)
class FloatLit(Exp):
    def __init__(self,v):
        self.value = v
    def accept(self,v): return v.visitFloatLit(self)

In [ ]:
from functools import reduce
class ASTGeneration(MPVisitor):
    def visitProgram(self,ctx:MPParser.ProgramContext):
        return self.visit(ctx.exp())

    def visitExp(self,ctx:MPParser.ExpContext):
        if ctx.getChildCount() == 1:
            return self.visit(ctx.term())
        return Binary(reduce(lambda acc,ele: Binary(':=',self.visit(ele),acc),ctx.term()[-2::-1],self.visit(ctx.term()[-1]))) 

    def visitTerm(self,ctx:MPParser.TermContext): 
        if ctx.getChildCount() == 3:
            return Binary(ctx.COMPARE().getText(),self.visit(ctx.factor()[0]),self.visit(ctx.factor()[1]))
        return self.visit(ctx.factor()[0])

    def visitFactor(self,ctx:MPParser.FactorContext):
        if ctx.getChildCount() == 1:
            return self.visit(ctx.operand()[0])
        return Binary(reduce(lambda acc,ele: Binary(ctx.ANDOR()[0].getText(),self.visit(ele),acc),ctx.operand()[-2::-1],self.visit(ctx.operand()[-1])))

    def visitOperand(self,ctx:MPParser.OperandContext):
        if ctx.getChildCount() == 3:
            return self.visit(ctx.exp())
        elif ctx.INTLIT(): 
            return IntLiteral(int(ctx.INTLIT().getText()))
        elif ctx.BOOLIT(): 
            return BooleanLiteral(bool(ctx.BOOLIT().getText()))
        else: return Id(ctx.ID().getText())

In [15]:
x1 = IntLit(1)
x2 = FloatLit(2.0)
x3 = BinExp(x1,"+",x1)
x4 = UnExp("-",x1)
x5 = BinExp(x4,"+",BinExp(IntLit(4),"*",x2))

In [19]:
v1 = Eval()
v2 = PrintPrefix()
v3 = PrintPostfix()
print(v3.visit(x5))
print(v2.visit(x1))
x5.accept(v2)

1 -. 4 2.0 * +
1


'+ -. 1 * 4 2.0'